# VideoNetClassification

Collaborators:

- Yahia Ehab
- Mariam Amr
- Mohamed Khaled

## Installation

In [1]:
!pip install -q imageio
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


## Imports

In [2]:
# @title Import the necessary modules
# TensorFlow and TF-Hub modules.
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed

logging.set_verbosity(logging.ERROR)

# Some modules to help with reading the UCF101 dataset.
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np

# Some modules to display an animation using imageio.
import imageio
from IPython import display

from urllib import request  # requires python3

## Data Loading

In [3]:
import pandas as pd

# Helper functions for the UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
_CACHE_DIR = tempfile.mkdtemp()
unverified_context = ssl._create_unverified_context()

def list_ucf_videos():
    """Lists videos available in UCF101 dataset."""
    global _VIDEO_LIST
    if not _VIDEO_LIST:
        index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
        videos = re.findall("(v_[\w_]+\.avi)", index)
        _VIDEO_LIST = sorted(set(videos))
    return list(_VIDEO_LIST)

def fetch_ucf_video(video):
    """Fetches a video and cache into local filesystem."""
    cache_path = os.path.join(_CACHE_DIR, video)
    if not os.path.exists(cache_path):
        urlpath = request.urljoin(UCF_ROOT, video)
        print("Fetching %s => %s" % (urlpath, cache_path))
        data = request.urlopen(urlpath, context=unverified_context).read()
        open(cache_path, "wb").write(data)
    return cache_path

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=30, resize=(224, 224)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if max_frames != 0 and len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames) / 255.0

def to_gif(images):
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave('./animation.gif', converted_images, duration=40)
    return embed.embed_file('./animation.gif')

# Define a function to create DataFrame with video paths and labels
def create_dataframe(num_videos=30):
    video_paths = []
    labels = []

    # List all UCF101 videos
    ucf_videos = list_ucf_videos()

    # Randomly select videos
    random_videos = random.sample(ucf_videos, num_videos)

    # Extract labels from video filenames
    for video in random_videos:
        label = video.split('_')[1]
        video_paths.append(fetch_ucf_video(video))
        labels.append(label)

    # Create DataFrame
    df = pd.DataFrame({'video_paths': video_paths, 'labels': labels})
    return df

# Create DataFrame with video paths and labels
df = create_dataframe()

# # Display the DataFrame
# print(df.head())


Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_SkyDiving_g02_c01.avi => /tmp/tmp4gm92x_d/v_SkyDiving_g02_c01.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_WalkingWithDog_g11_c01.avi => /tmp/tmp4gm92x_d/v_WalkingWithDog_g11_c01.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_BabyCrawling_g11_c01.avi => /tmp/tmp4gm92x_d/v_BabyCrawling_g11_c01.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_Nunchucks_g01_c04.avi => /tmp/tmp4gm92x_d/v_Nunchucks_g01_c04.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_HorseRiding_g12_c02.avi => /tmp/tmp4gm92x_d/v_HorseRiding_g12_c02.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_PoleVault_g20_c04.avi => /tmp/tmp4gm92x_d/v_PoleVault_g20_c04.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_PlayingDhol_g06_c02.avi => /tmp/tmp4gm92x_d/v_PlayingDhol_g06_c02.avi
Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_Swing_g13_c02.avi => /tmp/tmp4gm92x_d/v_Sw

In [4]:
import pandas as pd
# Save the DataFrame to a CSV file
df.to_csv('ucf101_videos_labels.csv', index=False)
#df = pd.read_csv('ucf101_videos_labels.csv')

In [5]:
df.head()

,video_paths,labels
0,/tmp/tmp4gm92x_d/v_SkyDiving_g02_c01.avi,SkyDiving
1,/tmp/tmp4gm92x_d/v_WalkingWithDog_g11_c01.avi,WalkingWithDog
2,/tmp/tmp4gm92x_d/v_BabyCrawling_g11_c01.avi,BabyCrawling
3,/tmp/tmp4gm92x_d/v_Nunchucks_g01_c04.avi,Nunchucks
4,/tmp/tmp4gm92x_d/v_HorseRiding_g12_c02.avi,HorseRiding


In [6]:
df['video_paths'][0]

'/tmp/tmp4gm92x_d/v_SkyDiving_g02_c01.avi'

### Load Video as GIF

Create `/GIFs` dir

In [7]:
frames_clip = [] # a '2d' array where each element is a group of frames corresponding to one video
for i in range(0, 20):
    # Load the first video from the DataFrame
    video_path = df['video_paths'][i]
    video = load_video(video_path)
    converted_video = np.clip(video*255, 0, 255).astype(np.uint8)
    frames_clip.append(converted_video)
    print("finished video: ", i)

# extract labels from the dataframe
labels = df['labels'].values


finished video:  0
finished video:  1
finished video:  2
finished video:  3
finished video:  4
finished video:  5
finished video:  6
finished video:  7
finished video:  8
finished video:  9
finished video:  10
finished video:  11
finished video:  12
finished video:  13
finished video:  14
finished video:  15
finished video:  16
finished video:  17
finished video:  18
finished video:  19


In [8]:
from tensorflow.keras.applications.inception_v3 import preprocess_input

preprocessed = []
for i in range(0, 20):
    preprocessed.append(preprocess_input(frames_clip[i]))
    print("finished video: ", i)

finished video:  0
finished video:  1
finished video:  2
finished video:  3
finished video:  4
finished video:  5
finished video:  6
finished video:  7
finished video:  8
finished video:  9
finished video:  10
finished video:  11
finished video:  12
finished video:  13
finished video:  14
finished video:  15
finished video:  16
finished video:  17
finished video:  18
finished video:  19


## Preprocessing

- CNN (InceptionV3 Model)
    1. Image Size should be 299*299 (only if we're using the full model)

- RNN
    1. LTSM

### CNN

In [9]:
# Load the InceptionV3 model from TensorFlow Hub
feature_extractor = hub.KerasLayer(
    "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4", trainable=False
)

In [47]:
# Function to extract features using InceptionV3
from tensorflow.keras.applications import InceptionV3

def extract_video_features(video_frames):
    base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
    video_features = []
    for video in video_frames:
        video_features.append(base_model.predict(video))
    return np.array(video_features)

video_features = extract_video_features(preprocessed)

1/1 [==============================] - 3s 3s/step


In [48]:
video_features[0]

array([[0.02520531, 0.16635466, 0.13847682, ..., 0.5598354 , 0.04727876,
        0.09656119],
       [0.13674994, 0.40171912, 0.05338402, ..., 0.25148198, 0.08858455,
        0.20174764],
       [0.15211831, 0.48810428, 0.11202975, ..., 0.02530198, 0.33688548,
        0.01934987],
       ...,
       [0.06045878, 0.7241143 , 0.1619721 , ..., 0.0111265 , 1.1488131 ,
        0.3639871 ],
       [0.22813064, 0.49462834, 0.19611564, ..., 0.07709448, 0.16632149,
        0.7587653 ],
       [0.43994546, 0.13805006, 0.13357519, ..., 0.07446862, 0.6358732 ,
        0.31239206]], dtype=float32)

In [49]:
split_index = int(0.8 * len(video_features))
train, test = video_features[:split_index], video_features[split_index:]
train_labels, test_labels = labels[:split_index], labels[split_index:20]
split = int(0.8*len(train))
train_features, validate_features = train[:split], train[split:]
train_labels, validate_labels = train_labels[:split], train_labels[split:]

In [50]:
train_features.shape

(12, 30, 2048)

In [13]:
max = 0
for i in range(0, len(train_features)):
  if train_features[i].shape[1] > max:
    max = train_features[i].shape[1]


In [54]:
for label in labels:
  print(label)

SkyDiving
WalkingWithDog
BabyCrawling
Nunchucks
HorseRiding
PoleVault
PlayingDhol
Swing
BalanceBeam
ParallelBars
BlowDryHair
ApplyEyeMakeup
ApplyLipstick
PlayingCello
SoccerPenalty
Diving
CliffDiving
BrushingTeeth
PlayingTabla
Skiing
Biking
JugglingBalls
JumpRope
Rafting
HandstandWalking
FloorGymnastics
FieldHockeyPenalty
Nunchucks
VolleyballSpiking
Bowling


In [71]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
label_encoder = LabelEncoder()
# Fit the LabelEncoder on the training labels
train_labels_encoded = label_encoder.fit_transform(train_labels)
# Transform the validation labels using the fitted LabelEncoder
validate_labels_encoded = label_encoder.fit_transform(validate_labels)
train_labels_onehot = to_categorical(train_labels_encoded)
validate_labels_onehot = to_categorical(validate_labels_encoded)
#train_labels_onehot = np.reshape(train_labels_onehot, (train_labels_onehot.shape[0], 30))
train_labels_onehot.ndim()
#validate_labels_onehot = np.reshape(validate_labels_onehot, (validate_labels_onehot.shape[0], 30))

TypeError: 'int' object is not callable

In [67]:
train_labels_onehot.shape

(12, 12)

In [62]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam

# RNN model creation
rnn_model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(30, 2048)),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(labels), activation='softmax')
])

rnn_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

rnn_model.fit(train_features, train_labels_onehot, validation_data=(validate_features, validate_labels_onehot), epochs=4)

Epoch 1/4


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    